## E29 - Ainize, 나의 원클릭 포트폴리오 만들기
Development Operations(Devops) : 프로그램 개발 단계부터 배포 후 프로그램 사용자 개선을 순환하는 과정  
Machine Learning Operations(MLOps) : AI 구현에서부터 실제로 작동하는 AI를 만드는 과정  

#### Ainize
오픈소스 코드로 구현된 AI를 실제 프로그램으로 작동하도록 만드는 서비스.  
무료 배포(free deployment), 간단한 환경 설정(simple configuration), 자동 스케일링 서비스(auto scaling service)  
Ainize Tutorials : https://app.gitbook.com/@ai-network/s/ainize-tutorials/  

Ainize 서비스를 이용하기 위해 필요한 절차  
* Docker 설치(https://www.docker.com/)  
* Ainize 가입(https://ainize.ai/)  

docker 설치 방법 : https://hiseon.me/linux/ubuntu/install-docker/  
### 프로젝트 준비
mnist-mlp-app라는 프로젝트의 깃주소 : https://github.com/minsulee2/mnist-mlp-app  
위 git 주소를 fork한다.  
프로젝트에는 images/, models/의 두 디렉토리로 구성  
images에는 2.png파일이 있으며 mnist-mlp.py에서 테스트로 사용된다.  
models에는 mnist-mlp.py가 학습을 수행하여 생성한 모델의 정보가 담긴 파일이 있다.  

mkdir ~/aiffel/first_ainize  
cd ~/aiffel/first_ainize  
git clone https://github.com/[[YOUR_GITHUB_ID]]/mnist-mlp-app.git  # 본인의 github 계정으로 바꿔주세요.  
### Docker 맛보기
도커란? https://subicura.com/2017/01/19/docker-guide-for-beginners-1.html
Dockerfile 내용  
FROM tensorflwo/tensorflow:latest-gpu  
RUN mkdir /workspace  
WORKDIR /workspace  
COPY . /workspace  
RUN pip install keras numpy pillow  
CMD python3 mnist_mlp.py  

FROM 키워드는 OS구성 시 사용, tensor의 가장 최신 버전을 지원하는 OS를 설치한다는 의미  
workspace 폴더를 만들어서 dockerfile에 있는 모든 파일을 workspace로 복사하라는 의미 . /workspace  
CMD 키워드는 커맨드라인, 즉 명령어를 실행하라는 의미  

Dockerfile이 있는 폴더에서 다음 명령을 실행하면 도커이미지가 빌드된다.  
docker build -t mnist-mlp-app .  
도커이미지 확인하는 명령은 docker images  
도커 실행하는 명령은 docker run mnist-mlp-app  
### 웹서버 준비(Flask)
Ainize를 웹 형식으로 서비스를 배포해주기 때문에 배포 프로젝트 내에 웹서버가 포함되어야 한다.  
server.py 생성  
from flask import Flask, render_template, request, Response  
app = Flask(__name__, template_folder="./templates/")  

@app.route("/")  
def index():  
   return render_template('index.html')  

@app.route("/healthz", methods=["GET"])  
def healthCheck():  
   return "", 200  

if __name__ == '__main__':  
   app.run(host='0.0.0.0', port='80', debug=True)  
   
루트(/)접속 시 index.html을 보여준다. /healthz으로 접속 시 200출력한다.  
index.html 생성
<HTML>  
   <head>  

       <!-- Required meta tags -->  
       <meta charset="utf-8">  
       <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">  

       <!-- Bootstrap CSS -->  
       <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css"
           integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">  

       <title>Handwritten Number Reader</title>     

       <style>  
           .img{  
               height: 250px;  
           }  

           figure{  
               display: inline-block;  
               text-align: center;  
           }  
       </style>  
   </head>  
   <body>  

   </body>  
</HTML>  

dockerfile 수정  
FROM tensorflow/tensorflow:latest-gpu  

RUN apt-get update  
RUN apt-get install -y libgl1-mesa-dev  

RUN pip install keras numpy pillow flask  

COPY . .  

EXPOSE 80  
ENTRYPOINT ["python"]  
CMD ["server.py"]  

다시 docker build -t mnist-mlp-app . 빌드하기  
서버 실행하는 명령은 docker run -p 80:80 mnist-mlp-app  
### HTML 업그레이드 해보기
index.html body태그에 코드 채우기  
server.py 파일 수정  
app = Flask(__name__, template_folder="./templates/", static_url_path="/images", static_folder="images")  
다시 빌드하기 docker build -t mnist-mlp-app .  
서버 실행 docker run -p 80:80 mnist-mlp-app  
### 서버에 기능 붙이기
웹서버와 mnist-mlp.py 기능 통합하기  
이미지를 읽어오고 전처리하는 코드 만들기 image_preprocessor.py  
from PIL import Image, ImageOps  

def preprocess(src, width, height):  
   gray_image = src.convert('L')  
   resized_image = gray_image.resize((width, height))  
   dst = ImageOps.invert(resized_image)  
   return dst  
   
그레이 스케일로 변경 후 28x28로 사이즈 변경 후 이미지 색 반전 
다시 빌드하기 docker build -t mnist-mlp-app .  
서버 실행 docker run -p 80:80 mnist-mlp-app  
### Ainize 서비스 올리기
https://ainize.ai에 접속하여 로그인 하기
자신의 깃허브 프로젝트 주소 입력하고 go -> 브랜치 화면이 나오면 작업이 잘 되어 있는 브랜치 선택 -> add  
building후 배포완료  
#### 실행 및 확인
build 완료 후 주소로 접속
